# Decision Trees: Theory and Practice
This notebook mirrors the markdown modules: intro, entropy_information_gain, gini_impurity, overfitting_pruning, math_behind, and adds a practical implementation.

## 1. Introduction
Decision Trees are non-parametric models for classification and regression.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.datasets import load_breast_cancer, make_classification
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

## 2. Entropy and Information Gain (Overview)
Entropy H = -Σ p log2 p. Information Gain = H(parent) - weighted child entropies.

## 3. Gini Impurity (Overview)
Gini = 1 - Σ p^2. CART maximizes decrease in Gini.

## 4. Fit a Decision Tree (Classification)

In [ ]:
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
clf = tree.DecisionTreeClassifier(random_state=42, criterion='gini', min_samples_leaf=5)
clf.fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

## 5. Visualize the Tree

In [ ]:
plt.figure(figsize=(16,10))
tree.plot_tree(clf, filled=True, feature_names=[f'f{i}' for i in range(X.shape[1])], class_names=['0','1'])
plt.show()

## 6. Hyperparameter Tuning and Pruning (ccp_alpha)

In [ ]:
param_grid = {
    'max_depth': [None, 4, 6, 8, 12],
    'min_samples_leaf': [1, 5, 10, 20],
    'ccp_alpha': [0.0, 0.0005, 0.001, 0.005, 0.01]
}
gs = GridSearchCV(tree.DecisionTreeClassifier(random_state=42), param_grid, cv=5, n_jobs=-1)
gs.fit(X_train, y_train)
print('Best params:', gs.best_params_)
print(classification_report(y_test, gs.best_estimator_.predict(X_test)))

## 7. Math Behind Splits (Demonstration)
We compute Gini decrease for a toy 1D dataset across thresholds.

In [ ]:
X1, y1 = make_classification(n_samples=60, n_features=1, n_redundant=0, n_informative=1, n_clusters_per_class=1, random_state=0)
x = X1[:,0]
order = np.argsort(x)
x, y1 = x[order], y1[order]
def gini(counts):
    n = counts.sum()
    if n==0: return 0.0
    p = counts / n
    return 1.0 - np.sum(p*p)
best = (-1, -1)
left = np.zeros(2)
right = np.bincount(y1, minlength=2).astype(float)
G_parent = gini(right)
for i in range(len(x)-1):
    cls = y1[i]
    left[cls]+=1
    right[cls]-=1
    if x[i]==x[i+1]:
        continue
    wL = (i+1)/len(x)
    wR = 1-wL
    score = G_parent - (wL*gini(left) + wR*gini(right))
    if score>best[0]:
        best = (score, 0.5*(x[i]+x[i+1]))
best